In [ ]:
papers['row_id']

,title,abstract,authorsids
row_id,,,
zyhxRc9bew,What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra..."
zyZkaqNnpa,Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa..."
zuXyQsXVLF,Enhancing Adversarial Contrastive Learning via...,Adversarial contrastive learning (ACL) is a te...,"[~Xilie_Xu1, ~Jingfeng_Zhang1, ~Feng_Liu2, ~Ma..."
ztDxO15N7f,An Optimization-based Approach To Node Role Di...,"Similar to community detection, partitioning t...","[~Michael_Scholkemper1, ~Michael_T_Schaub1]"
zsOOqjaj2z,Generator Identification for Linear SDEs with ...,"In this paper, we present conditions for ident...","[~Yuanyuan_Wang5, ~Xi_Geng1, ~Wei_Huang8, ~Biw..."
...,...,...,...
08hStXdT1s,Knowledge Diffusion for Distillation,The representation gap between teacher and stu...,"[~Tao_Huang5, ~Yuan_Zhang20, ~Mingkai_Zheng1, ..."
05P1U0jk8r,Exploiting hidden structures in non-convex gam...,A wide array of modern machine learning applic...,"[~Iosif_Sakos1, ~Emmanouil-Vasileios_Vlatakis-..."
02Uc0G2Cym,Robustness Guarantees for Adversarially Traine...,We study robust adversarial training of two-la...,"[~Poorya_Mianjy1, ~Raman_Arora1]"


In [2]:
reecrit_total

,abstract
0,Our framework uses the confusion density matri...
1,"Even when a perfect ""stable feature"" exists, s..."
2,By incorporating adversarial and standard inva...
3,Our work presents a novel method for partition...
4,This paper establishes generic identifiability...
...,...
3213,DiffKD explicitly denoises student features us...
3214,"Proposed PHGD, a first-order method using grad..."
3215,"By maximizing a lower bound on the $0/1$-loss,..."
3216,Optimal latent spaces for generative models ar...


In [13]:
import pandas as pd
import torch

papers=pd.read_json('submission_metadata.json').T
papers.index.name = 'row_id'

malicious=pd.read_json('101ranking.json')
authors=pd.read_json('authors92K.json')
reecrit_total=pd.read_csv('reecrit_TOTAL.csv')
reecrit_total['row_id']=list(papers.index)


papers_tensor=torch.load('papers.pt')
malicious_tensor=torch.load('malicious.pt')
authors_tensor=torch.load('authors.pt')
rewritten_tensor=torch.load('rewritten.pt')

rewritten_all_papers=torch.load('rewritten_all_papers.pt')
reecrit_total_tensor=rewritten_all_papers

print(authors_tensor.shape)
print(papers_tensor.shape)
print(malicious_tensor.shape)
print(rewritten_tensor.shape)
print(rewritten_all_papers.shape)
reecrit_total


torch.Size([92819, 768])
torch.Size([3218, 768])
torch.Size([300, 768])
torch.Size([300, 768])
torch.Size([3218, 768])


,abstract,row_id
0,Our framework uses the confusion density matri...,zyhxRc9bew
1,"Even when a perfect ""stable feature"" exists, s...",zyZkaqNnpa
2,By incorporating adversarial and standard inva...,zuXyQsXVLF
3,Our work presents a novel method for partition...,ztDxO15N7f
4,This paper establishes generic identifiability...,zsOOqjaj2z
...,...,...
3213,DiffKD explicitly denoises student features us...,08hStXdT1s
3214,"Proposed PHGD, a first-order method using grad...",05P1U0jk8r
3215,"By maximizing a lower bound on the $0/1$-loss,...",02Uc0G2Cym
3216,Optimal latent spaces for generative models ar...,00EKYYu3fD


In [14]:
from torch.nn.functional import cosine_similarity

def my_cosine_similarity(paper,list_of_papers): 
    """
    Calcul la similarité cosinus entre un papier et une les papiers d'un auteur
    """
    rez=[]
    for paper2 in list_of_papers:
        rez.append(cosine_similarity(paper,paper2,dim=0))
    return rez

def get_similarity(author, paper, mode='max'):
        similarities = my_cosine_similarity(paper,authors_dico[author])
        if mode== "max":
            return max(similarities)
        elif mode == "avg":
            return torch.stack(similarities).mean(dim=0)


In [15]:
i=0
authors_dico={}
for index,row in authors.iterrows():
    if(row['AuthorID'] not in authors_dico):
        authors_dico[row['AuthorID']]=[] #si l'auteur est pas dans le dico, on lui crée une liste vide
    authors_dico[row['AuthorID']].append(authors_tensor[i]) 
    i+=1

targets=[]
for elt in malicious['colluding_reviewer_profile_link']:
    targets.append(elt[34:])
malicious['target'] = targets

liste_ids=[]
for id, row in malicious.iterrows():
  for index,d in papers.iterrows():
    if(d['title']==row['title']):
      liste_ids.append(index)

malicious['id']=liste_ids

In [16]:
#il ne faut garder que les bons auteurs ! ceux qui ont été des targets.
good_authors={}
i=0
for id,elt in malicious.iterrows():
    if(elt['target'] in good_authors.keys()):
        print(i) #des auteurs sont targets plusieurs fois
        i+=1
    good_authors[elt['target']] = authors_dico[elt['target']]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [17]:
reecrit_total_tensor.shape

torch.Size([3218, 768])

In [30]:
dico_rankings={}
i=0
for author_id,author_tens in good_authors.items():
    dico_rankings[author_id]={}
    for paper_tens,paper_metadata in zip(reecrit_total_tensor,reecrit_total.iterrows()):
        dico_rankings[author_id][paper_metadata[1]['row_id']]=get_similarity(author_id,paper_tens,mode='max')
    i+=1
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [31]:
for key,value in dico_rankings.items():
    for author_id,author_tens in value.items():
        value[author_id]=float(author_tens) #transforme les tensors en floats psk c'est pas serialisable pour en faire un JSON
import json
with open("rewritten_rankings.json", "w") as f:
    json.dump(dico_rankings, f)

In [58]:
import json
with open("dico_rankings.json", "r") as f:
    my_dict = json.load(f)
with open("rewritten_rankings.json", "r") as f:
    my_dict2 = json.load(f)


my_dict=my_dict2

In [59]:
print(my_dict['~Adam_W_Harley1'])
print(my_dict2['~Adam_W_Harley1'])

{'zyhxRc9bew': 0.7545390725135803, 'zyZkaqNnpa': 0.7677703499794006, 'zuXyQsXVLF': 0.7355977296829224, 'ztDxO15N7f': 0.7089229822158813, 'zsOOqjaj2z': 0.7348286509513855, 'zrUEHZ6s9C': 0.7460786700248718, 'zrLxHYvIFL': 0.774827241897583, 'zrCmeqV3Sz': 0.7475934028625488, 'zqyVjCjhYD': 0.7059093713760376, 'zqOcW3R9rd': 0.7105420827865601, 'zq4vFneRiA': 0.8002848029136658, 'zpVCITHknd': 0.698144793510437, 'znudaK78u8': 0.8015661239624023, 'znY173SCxu': 0.7439345121383667, 'znW5jNIOED': 0.6700693964958191, 'zn5ihqknGj': 0.7130997776985168, 'zmWNe1V6jg': 0.6620471477508545, 'zkfyOkBVpz': 0.7523143291473389, 'zjpjsJeVJZ': 0.7759411931037903, 'zfHCKDzzC8': 0.6816799640655518, 'zfCNwRQ569': 0.6640022993087769, 'zdli6OxpWd': 0.737575113773346, 'zaQ7wV9NOg': 0.7566002011299133, 'zXckveawHa': 0.7901536226272583, 'zWxKYyW9ik': 0.6615030765533447, 'zW1uVN6Mbv': 0.7469622492790222, 'zUYfbdNl1m': 0.6888818144798279, 'zTSlm4nmlH': 0.8000134229660034, 'zQTi3pziFp': 0.7642422318458557, 'zQOYGDc9pu': 0.

In [60]:
for id,row in malicious.iterrows():
    my_dict[row['target']]['ORIGINAL']=my_dict[row['target']].pop(row['id'])

#for key,value in my_dict.items(): #On a remplacé cible ID par ORIGINAL
    #print(value['ORIGINAL'])

In [61]:
def find_name_rank(name,liste):
    for i,elt in enumerate(liste):
        if(name==elt[0]): 
            return i

In [62]:
id=0
for id,row in malicious.iterrows():
    similarity_score=get_similarity(row['target'],malicious_tensor[id],mode='avg')
    my_dict[row['target']]['MALICIOUS']=similarity_score
    similarity_score_rewritten=get_similarity(row['target'],rewritten_tensor[id],mode='avg')
    my_dict[row['target']]['REWRITTEN']=similarity_score_rewritten

In [63]:
dict_sorted={}
dict_sorted2={}
for key,value in my_dict.items():
    dict_sorted[key]=sorted(value.items(), key=lambda x: x[1], reverse=True)

for key,value in my_dict2.items():
    dict_sorted2[key]=sorted(value.items(), key=lambda x: x[1], reverse=True)

In [ ]:
#dict_sorted2['~Adam_W_Harley1']
#dict_sorted['~Adam_W_Harley1']


[('2ePf1sBgLU', 0.8404171466827393),
 ('jDIlzSU8wJ', 0.831592321395874),
 ('R6qMmdl4qP', 0.8290446400642395),
 ('KBXcDAaZE7', 0.8226078152656555),
 ('pQF9kbM8Ea', 0.8218309879302979),
 ('vF8ukt5l1R', 0.8214066028594971),
 ('mQPNcBWjGc', 0.8205826282501221),
 ('yC3q7vInux', 0.8175647258758545),
 ('i39yXaUKuF', 0.8161670565605164),
 ('pIXTMrBe7f', 0.8140406012535095),
 ('OIJ3VXDy6s', 0.81382155418396),
 ('HZQZli6amV', 0.8136555552482605),
 ('VpGFHmI7e5', 0.8134263157844543),
 ('7JuReDmGSL', 0.8103806376457214),
 ('NemifGnD2E', 0.8098581433296204),
 ('S1KGaTSOTS', 0.8097715973854065),
 ('eYCGrGdKf3', 0.8096655607223511),
 ('bbbbbov4Xu', 0.8081003427505493),
 ('HPrd17Qvbp', 0.8073820471763611),
 ('u8srPlinoj', 0.807363748550415),
 ('Grz2ijKrWI', 0.8073527216911316),
 ('TRuqrVsmZK', 0.8069823384284973),
 ('gUTVpByfVX', 0.8069465160369873),
 ('DVm0xxaEq1', 0.8063191771507263),
 ('l6ypbj6Nv5', 0.8052704334259033),
 ('cLQCCtVDuW', 0.8050909638404846),
 ('FasIQqsJhe', 0.8046282529830933),
 ('t1

In [64]:
moyenne_ORIGINAL=[]
moyenne_MALICIOUS=[]
moyenne_REWRITTEN=[]

for id,row in malicious.iterrows():
    a=find_name_rank('ORIGINAL',dict_sorted[row['target']])
    b=find_name_rank('MALICIOUS',dict_sorted[row['target']])
    c=find_name_rank('REWRITTEN',dict_sorted[row['target']])
    
    moyenne_ORIGINAL.append(a)
    moyenne_MALICIOUS.append(b)
    moyenne_REWRITTEN.append(c)


In [65]:
compteur=0
for i in range(0,len(moyenne_MALICIOUS)):
    if(moyenne_MALICIOUS[i]<=5):
        compteur+=1
print(compteur)

25


In [66]:
compteur=0
for i in range(0,len(moyenne_REWRITTEN)):
    if(moyenne_REWRITTEN[i]<=5):
        compteur+=1
print(compteur)

1


In [49]:
for a,b,c in zip(moyenne_ORIGINAL,moyenne_MALICIOUS,moyenne_REWRITTEN):
    pass
    #print(a,b,c)

ORIGINAL_MEAN=sum(moyenne_ORIGINAL)/len(moyenne_ORIGINAL)
MALICIOUS_MEAN=sum(moyenne_MALICIOUS)/len(moyenne_MALICIOUS)
REWRITTEN_MEAN=sum(moyenne_REWRITTEN)/len(moyenne_REWRITTEN)

In [50]:
ORIGINAL_MEAN-MALICIOUS_MEAN

209.3066666666667

In [54]:
REWRITTEN_MEAN

513.9766666666667

In [52]:
ORIGINAL_MEAN

260.4066666666667

In [53]:
MALICIOUS_MEAN

51.1

In [37]:
from scipy.stats import kendalltau
import numpy as np

taus = [kendalltau([x[0] for x in dict_sorted[k]], [x[0] for x in dict_sorted2[k]])[0] for k in dict_sorted]
print(f"Tau moyen: {np.mean(taus):.4f} ± {np.std(taus):.4f}")

Tau moyen: 0.0010 ± 0.0124


In [ ]:
dict_sorted['~Adam_W_Harley1']

[('2ePf1sBgLU', 0.8404171466827393),
 ('jDIlzSU8wJ', 0.831592321395874),
 ('R6qMmdl4qP', 0.8290446400642395),
 ('KBXcDAaZE7', 0.8226078152656555),
 ('pQF9kbM8Ea', 0.8218309879302979),
 ('vF8ukt5l1R', 0.8214066028594971),
 ('mQPNcBWjGc', 0.8205826282501221),
 ('yC3q7vInux', 0.8175647258758545),
 ('i39yXaUKuF', 0.8161670565605164),
 ('pIXTMrBe7f', 0.8140406012535095),
 ('OIJ3VXDy6s', 0.81382155418396),
 ('HZQZli6amV', 0.8136555552482605),
 ('VpGFHmI7e5', 0.8134263157844543),
 ('7JuReDmGSL', 0.8103806376457214),
 ('NemifGnD2E', 0.8098581433296204),
 ('S1KGaTSOTS', 0.8097715973854065),
 ('eYCGrGdKf3', 0.8096655607223511),
 ('bbbbbov4Xu', 0.8081003427505493),
 ('HPrd17Qvbp', 0.8073820471763611),
 ('u8srPlinoj', 0.807363748550415),
 ('Grz2ijKrWI', 0.8073527216911316),
 ('TRuqrVsmZK', 0.8069823384284973),
 ('gUTVpByfVX', 0.8069465160369873),
 ('DVm0xxaEq1', 0.8063191771507263),
 ('l6ypbj6Nv5', 0.8052704334259033),
 ('cLQCCtVDuW', 0.8050909638404846),
 ('FasIQqsJhe', 0.8046282529830933),
 ('t1

In [39]:
for key,value in dict_sorted.items():
    print(kendalltau(dict_sorted[key],dict_sorted2[key]))

SignificanceResult(statistic=np.float64(0.7483723185081361), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7519027104146352), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.752850498061641), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.744412463518025), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7490612728674402), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7524757605559658), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7512973293507176), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7578996030414297), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7507660689772491), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7520933567663383), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7528972198023779), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7537370590903387), pvalue

In [47]:
sorted1

[('0BfQT652sC', 0.8741604089736938),
 ('MCkUS1P3Sh', 0.8598865270614624),
 ('KUBFYAPdqN', 0.8531662225723267),
 ('ofa1U5BJVJ', 0.8509287238121033),
 ('tP50lLiZIo', 0.8483631610870361),
 ('wbg4JEM5Jp', 0.8479827046394348),
 ('vM5VnNQ4n7', 0.8453448414802551),
 ('Vbm5UCaYeh', 0.845332145690918),
 ('YeP8osxOht', 0.8445459008216858),
 ('Yc9bqbnrbs', 0.8425987958908081),
 ('c8nIdZ5HJJ', 0.8424784541130066),
 ('oi45JlpSOT', 0.8417927026748657),
 ('09bZyE9tfp', 0.8363831639289856),
 ('LjWJLkSpjh', 0.8358122706413269),
 ('VLnEFGu9V7', 0.8352081775665283),
 ('4mXYJzoPhf', 0.8348504900932312),
 ('vBHKSTgcYQ', 0.834486722946167),
 ('gaXAjtHic2', 0.8343656063079834),
 ('xF89MjFbWp', 0.8328521251678467),
 ('UPo8vlZ0wQ', 0.8306301832199097),
 ('gYetLsNO8x', 0.8299038410186768),
 ('FtZ7lUwH99', 0.8284971714019775),
 ('Ntd6X7uWYF', 0.8279247283935547),
 ('YiRX7nQ77Q', 0.8273543119430542),
 ('J3taqrzyyA', 0.8269066214561462),
 ('r6xGZ0XL2g', 0.8264058828353882),
 ('DqfdhM64LI', 0.8261038064956665),
 ('

In [ ]:
#A COMPLETEMENT RECHECK
from scipy.stats import kendalltau
import numpy as np

# Convert dict_sorted and dict_sorted2 into proper ranking arrays for Kendall's Tau
taus = []
p_values = []

for key in dict_sorted:
    # Get the sorted lists for this author
    sorted1 = dict_sorted[key]
    sorted2 = dict_sorted2[key]
    
    # Create a mapping from paper_id to rank (position in sorted list)
    # Position 0 = rank 1 (highest similarity), position 1 = rank 2, etc.
    rank1 = {paper_id: rank for rank, (paper_id, score) in enumerate(sorted1)}
    rank2 = {paper_id: rank for rank, (paper_id, score) in enumerate(sorted2)}
    
    # Get common paper IDs and sort them to ensure consistent ordering
    common_papers = sorted(set(rank1.keys()) & set(rank2.keys()))
    
    # Create aligned ranking arrays: array1[i] = rank of paper i in dict_sorted
    #                                 array2[i] = rank of paper i in dict_sorted2
    array1 = [rank1[paper_id] for paper_id in common_papers]
    array2 = [rank2[paper_id] for paper_id in common_papers]
    
    # Compute Kendall's Tau
    tau, p_value = kendalltau(array1, array2)
    taus.append(tau)
    p_values.append(p_value)

print(f"Kendall's Tau moyen: {np.mean(taus):.4f} ± {np.std(taus):.4f}")
print(f"Min: {np.min(taus):.4f}, Max: {np.max(taus):.4f}")
print(f"P-value moyen: {np.mean(p_values):.2e}")


Kendall's Tau moyen: 0.5609 ± 0.0548
Min: 0.3583, Max: 0.6796
P-value moyen: 2.15e-206


In [46]:
array2

[310,
 636,
 2509,
 795,
 2270,
 1456,
 3024,
 50,
 2038,
 2,
 954,
 1416,
 895,
 288,
 488,
 90,
 525,
 2108,
 1588,
 2243,
 2815,
 1149,
 2638,
 349,
 733,
 962,
 47,
 984,
 502,
 873,
 2803,
 2352,
 3095,
 837,
 2948,
 290,
 1707,
 2066,
 832,
 1301,
 1639,
 155,
 595,
 1580,
 805,
 2057,
 494,
 1840,
 720,
 1403,
 531,
 46,
 2708,
 2276,
 1244,
 3079,
 1644,
 54,
 2568,
 2418,
 2215,
 149,
 957,
 2469,
 2603,
 3182,
 245,
 2413,
 2475,
 1415,
 890,
 935,
 280,
 1476,
 2629,
 893,
 2260,
 1219,
 2592,
 1211,
 1482,
 1136,
 2847,
 1243,
 535,
 2059,
 126,
 2133,
 205,
 2734,
 1217,
 2483,
 2610,
 1885,
 2631,
 976,
 581,
 1965,
 1599,
 705,
 1769,
 1238,
 2869,
 864,
 98,
 2616,
 1944,
 2807,
 2627,
 2439,
 2295,
 2624,
 3113,
 1933,
 490,
 2363,
 2636,
 1942,
 55,
 2609,
 2259,
 2312,
 2210,
 1388,
 2546,
 3124,
 1712,
 981,
 1154,
 1314,
 2950,
 1858,
 230,
 2585,
 1730,
 1250,
 2658,
 1496,
 316,
 798,
 987,
 1239,
 2167,
 1603,
 2084,
 77,
 1585,
 343,
 2833,
 1613,
 2516,
 3170,

In [45]:
array1

[211,
 1409,
 2585,
 847,
 1960,
 1013,
 2543,
 12,
 2259,
 0,
 1342,
 1249,
 1890,
 2057,
 1502,
 97,
 555,
 2554,
 2081,
 2411,
 3207,
 922,
 2391,
 348,
 1072,
 766,
 57,
 1350,
 1288,
 1154,
 3147,
 2118,
 2789,
 1016,
 3050,
 221,
 1741,
 1348,
 678,
 977,
 2493,
 159,
 557,
 1698,
 966,
 3151,
 1121,
 1994,
 1545,
 1584,
 423,
 64,
 2519,
 2215,
 1395,
 2455,
 1700,
 139,
 2336,
 3169,
 2092,
 144,
 1279,
 2545,
 2973,
 3197,
 265,
 2703,
 1190,
 2223,
 501,
 70,
 141,
 2326,
 2544,
 1088,
 987,
 2896,
 1953,
 2227,
 2126,
 654,
 2721,
 1212,
 283,
 2494,
 142,
 1626,
 207,
 2782,
 1473,
 2969,
 2696,
 2213,
 2823,
 650,
 392,
 1699,
 2166,
 1722,
 1313,
 1975,
 1883,
 852,
 204,
 2849,
 2154,
 3182,
 1720,
 1997,
 1838,
 2596,
 3175,
 1644,
 761,
 1884,
 2871,
 2786,
 148,
 2845,
 2381,
 2262,
 2846,
 627,
 2834,
 2932,
 911,
 648,
 2121,
 2233,
 2576,
 1567,
 45,
 3051,
 2078,
 749,
 3095,
 723,
 373,
 533,
 1697,
 1439,
 1921,
 2237,
 856,
 167,
 1215,
 226,
 2693,
 1304,
 250